In [1]:
import random
from random import shuffle

from PIL import Image, ImageFilter
from IPython import display

from matplotlib.pyplot import imshow
%matplotlib inline

import pickle
import time

from sampling_utils import *
from gen_utils import *
from config_db import ConfigDB

In [2]:
num_pics = 10

min_num_cars = 1
max_num_cars = 2

domains = [[52,72],[1,19],[0,1],[0.35,1],[0.5,1.5],[0.25,1],[0.5,1],[0.5,1]]
types = ['int','int','float','float','float','float','float','float']

gen_misclass = True
store_config_db = True
store_err_tab = False

IOMIN_THRESH = 0.5
IOU_THRESH = 0.5

if gen_misclass:
    checkpoint = '/home/xyyue/checkpoint/final/model.ckpt-5000'
    import squeezedet as nn
    net = nn.init(checkpoint)

INFO:tensorflow:Summary name mean iou is illegal; using mean_iou instead.
INFO:tensorflow:Restoring parameters from /home/xyyue/checkpoint/final/model.ckpt-5000


In [ ]:
# Init data structure
if store_err_tab:
    err_table = error_table()
if store_config_db:
    config_db = ConfigDB()
    
r = 0

for j in range(min_num_cars,max_num_cars):
    
    path_data_set = './data/test/'
    #path_data_set = './data/misclass_2/' + str(j) + '/'
    #path_data_set = './data/misclass_err_table/' + str(j) + '/'
    
    
    i = 0
    v = 0

    while i < num_pics:

        # Sample configuration
        #num_cars = random.randint(min_num_cars, max_num_cars)
        num_cars = j
        sample_dims = [1,num_cars,num_cars,num_cars,1,1,1,1]
        sample, norm_sample = get_sample(v, 'random', types=types, domains=domains, dims=sample_dims)
        sample[3].sort(reverse=True)
        norm_sample[3].sort(reverse=True)
        
        good_sample = True
        # Check sample distance from stored ones
        if store_config_db:
            
            sample_pad = pad_sample(sample)
            good_sample = config_db.eps_dist(sample_pad,0.5)
            
            if good_sample:
                config_db.add(sample_pad)            
            else:
                #print('Sample rejected')
                r += 1
            
        if good_sample:
            # Generate image from sample
            img, ground_boxes, img_info = gen_image(sample)

            # Is it a good picture? (cars don't overlap too much)
            save_img = True
            for m in range(len(ground_boxes)):
                for n in range(m+1, len(ground_boxes)):
                    #print(iomin(ground_boxes[m], ground_boxes[n]))
                    if iomin(ground_boxes[m], ground_boxes[n]) > IOMIN_THRESH:
                        save_img = False

            # Check misclassification
            if gen_misclass and save_img:
                # Save tmp image    
                tmp_img_file_name = './tmp.png'
                img.save(tmp_img_file_name)

                # Classify img
                (boxes,probs,cats) = nn.classify(tmp_img_file_name,net) 

                save_img = False

                # Not cars
                if any(cats):
                    save_img = True
                else:
                    all_good_boxes = True
                    for ground_box in ground_boxes:
                        found_good_box = False
                        for box, prob in zip(boxes, probs):    
                            found_good_box = found_good_box or ((iou(ground_box, box) > IOU_THRESH) and prob > 0.5)
                        all_good_boxes = all_good_boxes and found_good_box
                    save_img = not(all_good_boxes)

            if save_img:
                
                print(dist)
                
                file_name = 'm_' + str(j) + '_' + str(i).zfill(6)

                # Save data
                save_image(img, file_name, path_data_set)
                save_label(ground_boxes, file_name, path_data_set)
                
                if store_err_tab:
                    err_table.update_with_elem(img_info)
                    
                i+=1
            v+=1

        if(v %50 == 0):
            print('i: ' + str(i) + ' ' + 'v: ' + str(v))
            

if store_err_tab:
    err_table_file = open(path_data_set + 'error_table' + str(),'w')
    pickle.dump(err_table,err_table_file)
    err_table_file.close()
    
if store_kd_tree:
    kd_tree_file = open(path_data_set + 'kd_tree' + str(),'w')
    pickle.dump(tree,kd_tree_file)
    kd_tree_file.close()

In [4]:
a = [[71], [7], [0.6481259501545733], [0.35889551518785573], [1.4120744939590137], [0.9189374354473185], [0.9776240024504621], [0.9597956849561484]]
b = [[64], [10], [0.34110162676566236], [0.8149413988886354], [0.9154506918197423], [0.4865547378397377], [0.8126097677475952], [0.7856083382532069]]
c = [[53], [2], [0.9299863883615281], [0.43904384546350306], [1.3873940098511786], [0.4861567462554541], [0.8768052257671992], [0.8603307951039953]]

af = config_db.pad_config(a)
bf = config_db.pad_config(b)
cf = config_db.pad_config(c)

config_db.dist(af,bf)

1
2
3.00069004126


3.0006900412619917

In [17]:
picul

0.062318786925770857

In [14]:
r

1046